In [33]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from time import time, localtime, strftime
import openpyxl
from openpyxl.styles import Font, PatternFill
from openpyxl.drawing.image import Image
import json
import os
from collections import defaultdict


In [1]:
class ScreenShotTaker:
    def __init__(self, workbook, sheet_name):
        self.workbook = openpyxl.load_workbook(filename=workbook, data_only=True)
        self.sheet = self.workbook[sheet_name]

        self.options = webdriver.ChromeOptions()
        self.options.add_argument("headless")
        self.options.add_argument("--window-size=1920,1440")
        self.options.add_argument(f'user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36')
        
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=self.options)
        
        self.network_map = {
            'ETH': "https://etherscan.io/tx/",
            'ERC20': "https://etherscan.io/tx/",
            "BTC": "https://www.blockchain.com/btc/tx/",
            "TERRA": "https://finder.terra.money/classic/tx/"
        }

    def __del__(self):
        self.driver.quit()

    def logger(func):
        def wrapper(*args, **kwargs):
            start = time()
            start_time = strftime("%Y-%m-%d %I:%M:%S", localtime(start))
            print(f"starts now! {start_time}")
            func(*args, **kwargs)
            end = time()
            end_time = strftime("%Y-%m-%d %I:%M:%S", localtime(end))
            print(f"completed at {end_time}, took {round(end - start, 2)} seconds")
        return wrapper
    
    @logger
    def get_screenshot(self, target_network, start = 1):
        for i in range(start, self.sheet.max_row + 1):
            network = self.sheet[f"K{str(i)}"].value
            if network in target_network:
                tx_hash = self.sheet[f"L{str(i)}"].value
                print(f"network: {network}, tx_hash: {tx_hash}")
                self.driver.implicitly_wait(6)
                self.driver.get( self.network_map[network] + tx_hash )
                self.driver.implicitly_wait(6)
                self.driver.save_screenshot(f"screenshots/{self.sheet_name}_{network}_row{i}.png")
                self.driver.implicitly_wait(6)

In [ ]:
filename ="screenshots/OKEx Funding Account History_v5.xlsx"
es = ScreenShotTaker(filename, "OKEx Funding 출금 History")
es.get_screenshot(["ERC20", "BTC"], 1)

In [ ]:
class XLSLoader:
    def __init__(self):
        self._WIDTH_FUDGE = (10/75)
        self._HEIGHT_FUDGE = (3/4)
        self._BASE_CELL_HEIGHT_IN_PIXEL = 18  
        self._BASE_CELL_WIDTH_IN_PIXEL = 75

        self.A_CELL_WIDTH = 142 * self._WIDTH_FUDGE
        self.sheet.column_dimensions['A'].width = self.A_CELL_WIDTH

        self.color_map = {
            "gray": PatternFill('solid', 'c8c8c8'),
            "green": PatternFill('solid', '80E12A'),
            "orange": PatternFill('solid', 'ffaa28'),
            "blue": PatternFill('solid', '9bc3ff'),
        }

        self.bold_map = {
            "black" :  Font(color="000000", bold=True, size = 12)  , 
            "white" :  Font(color="FFFFFF", bold=True, size = 14)
        }


In [35]:
class ScreenShotSaver:
    def __init__(self, filename, sheet_name):
        self.workbook = openpyxl.load_workbook(filename=filename, data_only=True)
        self.sheet = self.workbook.create_sheet(sheet_name)

        self._WIDTH_FUDGE = (10/75)
        self._HEIGHT_FUDGE = (3/4)
        self._BASE_CELL_HEIGHT_IN_PIXEL = 18  
        self._BASE_CELL_WIDTH_IN_PIXEL = 75

        self.A_CELL_WIDTH = 142 * self._WIDTH_FUDGE
        self.sheet.column_dimensions['A'].width = self.A_CELL_WIDTH

        self.color_map = {
            "gray": PatternFill('solid', 'c8c8c8'),
            "green": PatternFill('solid', '80E12A'),
            "orange": PatternFill('solid', 'ffaa28'),
            "blue": PatternFill('solid', '9bc3ff'),
        }

        self.bold_map = {
            "black" :  Font(color="000000", bold=True, size = 12)  , 
            "white" :  Font(color="FFFFFF", bold=True, size = 14)
        }

    def get_json_from(self, filename):
        with open(f"{filename}", "r") as f:
            json_data = json.load(f)
        return json_data

    def save_in(self, filename):
        self.workbook.save(f"{filename}")

    def _paint_background(self, range, color):
        for cell in self.sheet[range][0]:
            cell.fill = color

    def _write_date(self, row, datetime):
        self.sheet[f"A{row}"].value = datetime
        self.sheet[f"A{row}"].font = self.bold_map['white']
        self._paint_background(f"A{row}:L{row}", self.color_map['gray'] )

    def _write_header(self, row, string = "Terra Anchor Deposit"):
        self.sheet[f"A{row}"].value = string
        self.sheet[f"A{row}"].font = self.bold_map['black']
        self._paint_background(f"A{row}:L{row}", self.color_map['green'] )
        
    def _put_image(self, row, image_dir):
        image = Image(f'screenshots/peertec_anchor_deposit/{image_dir}.png')
        image.height = self._BASE_CELL_HEIGHT_IN_PIXEL * 40 / self._HEIGHT_FUDGE
        image.width = (142 + 75 * 11) * (21.1 / 19)
        self.sheet.add_image(image, f"A{row}")

    def _write_description_header(self, row, string = "Description"):
        self.sheet[f"A{row}"].value = string
        self.sheet[f"A{row}"].font = self.bold_map['black']
        self._paint_background(f"A{row}:L{row}", self.color_map['orange'] )

    def _write_description(self, row, datetime, value):
        self.sheet[f"a{row}"].value = f"{datetime} \n Peertec Terra 지갑에서 \n Anchor Market으로 {value} USTC 입금"
        self.sheet[f"a{row}"].font = self.bold_map['black']
        self._paint_background(f"A{row}:L{row}", self.color_map['blue'] )
        self.sheet.row_dimensions[row].height = self._BASE_CELL_HEIGHT_IN_PIXEL * 6 

    def start_painting(self, json_data):
        row = 1
        for each in json_data:
            self._write_date(row, each['datetime'].split()[0])
            self._write_header(row+1)
            self._put_image(row+2, each['datetime'])
            self._write_description_header(row+42)
            self._write_description(row+43, each['datetime'], each['value'])
            row += 45

In [72]:
file_name = "테라스샷 tmp.xlsx"
sheet_name = "tmp"
sss = ScreenShotSaver(file_name, sheet_name)
# 여기 json은 fromtx에서 가져오는 것
json_data = sss.get_json_from("anchor_deposit/Peertec Terra Anchor deposit.json")
sss.start_painting(json_data)
sss.save_in(f"테라스샷 tmp.xlsx")

In [ ]:
#TODO 상속클래스, 추상클래스도 적용가능

In [82]:
class MonthlyBalanceSnapshotSaver:
    def __init__(self, filename, sheet_name):
        self.workbook = openpyxl.load_workbook(filename=filename, data_only=True)
        self.sheet = self.workbook.create_sheet(sheet_name)

        self._WIDTH_FUDGE = (10/75)
        self._HEIGHT_FUDGE = (3/4)
        self._BASE_CELL_HEIGHT_IN_PIXEL = 18  
        self._BASE_CELL_WIDTH_IN_PIXEL = 75

        self.A_CELL_WIDTH = 142 * self._WIDTH_FUDGE
        self.sheet.column_dimensions['A'].width = self.A_CELL_WIDTH

        self.color_map = {
            "gray": PatternFill('solid', 'c8c8c8'),
            "green": PatternFill('solid', '80E12A'),
            "orange": PatternFill('solid', 'ffaa28'),
            "blue": PatternFill('solid', '9bc3ff'),
        }

        self.bold_map = {
            "black" :  Font(color="000000", bold=True, size = 12)  , 
            "white" :  Font(color="FFFFFF", bold=True, size = 14)
        }


    def save_in(self, filename):
        self.workbook.save(f"{filename}")
    
    def _paint_background(self, range, color):
        for cell in self.sheet[range][0]:
            cell.fill = color

    def _write_exchange_name(self, row, exchange_name):
        self.sheet[f"A{row}"].value = exchange_name
        self.sheet[f"A{row}"].font = self.bold_map['white']
        self._paint_background(f"A{row}:L{row}", self.color_map['gray'] )

    def _write_title(self, row, title):
        self.sheet[f"A{row}"].value = title
        self.sheet[f"A{row}"].font = self.bold_map['black']
        self._paint_background(f"A{row}:L{row}", self.color_map['green'] )
        
    def _put_image(self, row, image_dir):
        image = Image(f'snapshots/{image_dir}')
        image.height = self._BASE_CELL_HEIGHT_IN_PIXEL * 40 / self._HEIGHT_FUDGE
        image.width = (142 + 75 * 11) * (21.1 / 19)
        self.sheet.add_image(image, f"A{row}")
    
    def _paint_profile(self, row, exchange_name, title, image_dir):
        self._write_exchange_name(row, exchange_name)
        self._write_title(row+1, title)
        self._put_image(row+2, image_dir)

    def _paint_balance(self, row, image_dir):
        self._put_image(row, image_dir)
  
    def get_snapshots_map_from(self, snapshots):
        snapshots_map = defaultdict(list)
        for snapshot in snapshots:
            exchange_name = snapshot.split()[0]
            snapshots_map[exchange_name].append(snapshot)
        return snapshots_map    
    
    def start_painting(self, snapshots):
        row = 1
        num_pic = 0
        snapshots_map = self.get_snapshots_map_from(snapshots)
        for snapshots_list in snapshots_map.values():
            for snapshot in snapshots_list:
                is_profile = True if snapshot[-5] == '0' else False
                exchange_name = snapshot.split(" ")[0]
                title = exchange_name + " Profile" if is_profile else exchange_name + " Balance"

                # 밸런스의 개수가 있어야 하는데...
                # 딕셔너리를 쓸수밖에없게하는
                if is_profile:
                    self._paint_profile(row, exchange_name, title,  snapshot)
                    # 숫자안맞음
                    row += 43

                # 지갑같은 경우, 밸런스만 나오는데 밸런스만 나올 수 있게
                # value의 len == 1

                else:
                    if snapshot[-5] == "1":
                        self._write_title(row-1, title) 
                    # 여기서 row를 +43씩 해줘야 하는것
                    self._paint_balance(row, snapshot)
                        # 요거는 하나씩 하는 것
                    row += 41
                num_pic += 1
            row += 1
        print(num_pic)
        # print("폴더 안의 모든 이미지가 성공적으로 입력됨" if num_pic == len(snapshots) - 1 else "누락된 이미지 존재")
        

In [83]:
file_name = "월말 밸런스.xlsx"
sheet_name = "6월 월말 밸런스"
mbss = MonthlyBalanceSnapshotSaver(file_name, sheet_name)
snapshots = os.listdir('snapshots')
snapshots.sort()
print(len(snapshots)-1)
# .DS_store라는 숨김파일이 있음
mbss.start_painting(snapshots[1:])
mbss.save_in(f"월말 밸런스.xlsx")

41
41
